In [1]:
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt
import math
import random
from collections import defaultdict
import keras
from models import C3D
from keras import backend as K

Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 16, 224, 224, 3)   0         
_________________________________________________________________
conv3d_1 (Conv3D)            (None, 16, 224, 224, 16)  1312      
_________________________________________________________________
batch_normalization_1 (Batch (None, 16, 224, 224, 16)  64        
_________________________________________________________________
activation_1 (Activation)    (None, 16, 224, 224, 16)  0         
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 8, 112, 112, 16)   0         
_________________________________________________________________
conv3d_2 (Conv3D)    

In [38]:
class dataGenerator(keras.utils.Sequence):

    def __init__(self, filepath, batch_size, ffpath, segments = 16, test=False):
        self.filenames = list()
        self.labels = list()
        self.batch_size = batch_size
        self.filepath = filepath
        self.ffpath = ffpath
        
        with open(self.filepath,"r") as f:
            for line in f.readlines():
                arr = line.split(" ")
                self.filenames.append(arr[0])
                self.labels.append(int(arr[1].strip()))
        
    def __len__(self):
        return len(self.filenames)//self.batch_size

    def __getitem__(self, idx):
        batch_x = self.filenames[idx * self.batch_size:min((idx + 1) * self.batch_size, len(self.filenames))]
        batch_y = self.labels[idx * self.batch_size:min((idx + 1) * self.batch_size, len(self.filenames))]

        Xframes = list()
        Y = list()
        for index,each in enumerate(batch_x):
            infopath = os.path.join(self.ffpath,each,"info.txt")
            imgpath = os.path.join(self.ffpath,each,"frames")
            f = open(infopath,"r")
            total_frames = int(f.readlines()[0].strip().split(':')[1])
            f.close()
            idxs = sorted(np.random.randint(0, total_frames, 16))
            Xframes.append(self.getFrames(idxs, imgpath))
            Y.append(batch_y[index])

        finalX, finalY = np.array(Xframes), self.one_hot_encode(np.array(Y))
        return (finalX,finalY)
    
    def one_hot_encode(self,data, classes = 101):
        """
        :param data: data to be one hot encoded
        :return: np array with one hot encoding
        """
        labels = np.zeros((data.size, classes))
        labels[np.arange(data.size), data - 1] = 1
        return labels
    
    def readImg(self,path):
        img = cv2.imread(path)
        grayimg = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
        return grayimg
    
    def getFrames(self,idxs, imgpath):
        stack = list()
        for i in idxs:
            framename = "frame_"+str(i)+".jpg"
            stack.append(self.readImg(os.path.join(imgpath,framename)))
        return np.array(stack)

In [39]:
# folderpath = "../ucfTrainTestlist"
# datapath = "../../UCF-101"
# filename = "trainlist01.txt"
# dg = dataGeneratorNew(datapath,os.path.join(folderpath,filename),batch_size = 128)



folderpath = "../ucfTrainTestlist"
filenameTrain = "custom2Train.txt"
filenameVal = "custom2Val.txt"

filepath = os.path.join(folderpath,filenameTrain)
ffpath = os.path.join("../../VidRecognizer_old/FramesFlows/custom2")
dgTrain = dataGenerator(filepath,16,ffpath)

filepath = os.path.join(folderpath,filenameVal)
ffpath = os.path.join("../../VidRecognizer_old/FramesFlows/custom2")
dgVal = dataGenerator(filepath,16,ffpath)

In [40]:
#Create and Compile model
K.clear_session()
model = C3D()
model.compile(optimizer= keras.optimizers.Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
#Checkpointing

filepath="../model_checkpoints/c3d-weights-improvement-{epoch:02d}-{val_acc:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
es = EarlyStopping(monitor='val_acc', mode='max', min_delta=1, patience = 10)
callbacks_list = [checkpoint,es]

In [41]:
np.random.seed(0)
#Fit generator
history = model.fit_generator(dgTrain, epochs = 1, validation_data = dgVal, callbacks = callbacks_list)

Epoch 1/1
30/30 [==============================] - 55s 2s/step - loss: 4.7659 - acc: 0.0167 - val_loss: 4.3259 - val_acc: 0.0446
